### Graphs!  

actually similar to trees

<b> 127. Word Ladder </b>

only change from word to words that have exactly one letter difference. in this way the problem can be transformed into a graph problem, begin word => visit its neighbors (defined as word with only one char difference) => non-visited neighbors' neighbors => until we arrived at the end word.

so it is a breath first search structure. (BFS)

In [3]:
from collections import deque
class Solution:
    def ladderLength(self, beginWord, endWord, wordList):
        def construct_neighbors(word_list):
            d = {}
            for word in word_list:
                for i in range(len(word)):
                    # no need to worry i+1 is out of bound, cz slicing wont give index out of bound error
                    key = word[:i]+'_'+word[i+1:]
                    d[key] = d.get(key, [])+[word]  # {'_ot': ['lot', 'dot', 'hot']}
            return d
        
        def bfs_words(begin, end, word_dict):
            # queue has word and level => step
            # why initiate the step as 1? => example "hit" -> "hot" -> "dot" -> "dog" -> "cog", result should be 5, not 4
            queue, visited = deque([(begin, 1)]), set()
            while queue:
                word, step = queue.popleft()  # this is why using deque
                if word not in visited:  # this check makes the execution much much faster
                    visited.add(word)
                    if word == end:
                        return step
                    # otherwise find its neighbors and add to queue
                    for i in range(len(word)):
                        s = word[:i] + '_' + word[i+1:]
                        neighbors = word_dict.get(s, [])
                        for n_word in neighbors:
                            if n_word not in visited:
                                queue.append((n_word, step+1))
            return 0
        
        if endWord not in wordList:
            return 0
        word_dict = construct_neighbors(set(wordList))
        return bfs_words(beginWord, endWord, word_dict)
            
            
            
            
            

In [4]:
beginWord = "hit"
endWord = "cog"
word_dict = ["hot","dot","dog","lot","log","cog"]
a= Solution()
a.ladderLength(beginWord, endWord, word_dict)

5

bidirectional BFS. Search both from start word and end word, if they meet at some intermediate word, that means existing such a road from start word to end word

In [9]:
from collections import deque, defaultdict
class Solution:
    def ladderLength(self, beginWord, endWord, wordList):
        """bidirectional BFS"""
        
        def find_words(queue, one_visited, two_visited):
            """
            queue: [(word, level)], deque
            one_visited: {word: level} current direction
            two_visited; {word: level} the other direction
            """
            word, step = queue.popleft()
            for i in range(L):
                intermediate = word[:i]+'_'+word[i+1:]
                
                for neighbor in neighbors_dict[intermediate]:
                    if neighbor in two_visited:
                        return step + two_visited[neighbor]
                    if neighbor not in one_visited:
                        one_visited[neighbor] = step + 1
                        queue.append((neighbor, step+1))
            return
        
        if endWord not in wordList:
            return 0
        
        L = len(beginWord)
        
        # construct neighbors dictionary
        neighbors_dict = defaultdict(list)
        for word in wordList:
            for i in range(L):  # only need to loop L time
                key = word[:i]+'_'+word[i+1:]
                neighbors_dict[key].append(word)
        
        queue_begin = deque([(beginWord, 1)])
        queue_end = deque([(endWord, 1)])
        visited_begin = {beginWord: 1}
        visited_end = {endWord: 1}
        # if one queue is empty the other is not, but they still dont meet, that means no such a way
        while queue_begin and queue_end:
            # queue_begin, queue_end, visited_begin, visited_end are all reference and are updated in side of the
            # execution of find_words() method
            # go forward one level from begin word
            res = find_words(queue_begin, visited_begin, visited_end)
            if res:
                return res
            # go forward one level from end word
            res = find_words(queue_end, visited_end, visited_begin)
            if res:
                return res
        return 0

In [10]:
beginWord = "hit"
endWord = "cog"
wordList = ["hot","dot","dog","lot","log","cog"]
b= Solution()
b.ladderLength(beginWord, endWord, wordList)

5

<b> 126 Word Ladder II </b>

<b> 207 Course Schedule </b>

the problem is also known as topological sort problem, which is to find a global order for all nodes in a DAG (Directed Acyclic Graph) with regarding to their dependencies.

<b>Topological sort </b>  
construct a graph:{prerequisite: [dependings]}  
construct a degree dictionary: {course: degree integer}: n degree means this course has n prerequisits need to take  
the course with degree 0 is the one able to take first. if no course has 0 degree, the courses are in a loop, cant be finished.
first starting with courses with degree 0, find its depending courses and reduce their degrees by 1. that means number of requisites needs to be finished before able to take the depending course is reduced by 1. If now a new course has degree 0, add it to the start course queue.  
at the end if all courses are taken, each of the degree is 0, sum is 0 too

In [11]:
from collections import defaultdict, deque
def canFinish(numCourses, prerequisites):
    # construct a graph:{prerequisite: [dependings]}
    graph = defaultdict(list)
    # construct a degree dictionary: {course: degree integer}
    # n degree means this course has n prerequisits need to take
    degree = [0]*numCourses
    for d, pre in prerequisites:
        graph[pre].append(d)
        degree[d]+=1 # course number happens to be used as index

    # the course with degree 0 is the able to take first. if no course has 0 degree, the courses are in a loop, cant be finished
    start_courses = deque([i for i in range(numCourses) if degree[i]==0])
    while start_courses:
        pre = start_courses.popleft()
        for j in graph[pre]:
            # j is the depending courses of i
            degree[j]-=1  # number of requisites needs to be finished before able to take the course j is reduced by 1
            if degree[j]==0:
                # able to take it as next course
                start_courses.append(j)
    # if all courses are taken, they will all be 0
    return not sum(degree)

In [12]:
numCourses = 2
prerequisites = [[1,0],[0,1]]
canFinish(numCourses, prerequisites)

False

<b> 210 course schedule II </b>.   
similar as above

In [14]:
from collections import defaultdict, deque
def findOrder(numCourses, prerequisites): 
    graph = defaultdict(list)  # {prerequisite: [depending courses]}
    degree=[0]*numCourses  # [integer] how many prerequisites needs to take

    for course, pre in prerequisites:
        graph[pre].append(course)
        degree[course]+=1 

    result=[]
    # start with course with no prerequisite
    starting_course = deque([i for i in range(numCourses) if degree[i]==0])
    while starting_course:
        course = starting_course.popleft()
        result.append(course)
        # its dependencies' degree - 1
        for d in graph[course]:
            degree[d]-=1
            if degree[d]==0:
                # able to be taken next
                starting_course.append(d)

    if sum(degree)==0:
        # all courses are taken
        return result
    else:
        return []

In [15]:
numCourses = 4
prerequisites=[[1,0],[2,0],[3,1],[3,2]]
findOrder(numCourses, prerequisites)

[0, 1, 2, 3]